In [5]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Set the path to your dataset
data_dir = r'C:\Users\Saurav\Documents\predictive analytics project'

# Define image size and batch size
IMG_SIZE = 256
BATCH_SIZE = 32
NUM_CLASSES = 21

# Define data augmentation and normalization
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Load training data
train_data = datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

# Load validation data
val_data = datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)


ModuleNotFoundError: No module named 'tensorflow'

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),

    Dense(256, activation='relu'),
    Dropout(0.5),

    Dense(NUM_CLASSES, activation='softmax')  # Output layer with 21 classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


ModuleNotFoundError: No module named 'tensorflow'

In [5]:
# Train the model
history = model.fit(train_data, validation_data=val_data, epochs=10)


Epoch 1/10


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 3), output.shape=(None, 21)

In [7]:
print(train_data.class_indices)


{'.ipynb_checkpoints': 0, 'ChangePrediction': 1, 'dataset': 2}


In [9]:
NUM_CLASSES = len(train_data.class_indices)  # Dynamically set the number of classes


In [11]:
model.add(Dense(NUM_CLASSES, activation='softmax'))  # Ensure NUM_CLASSES is set correctly


In [13]:
train_data = datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',  # This ensures one-hot encoding
    subset='training'
)


Found 16800 images belonging to 3 classes.


In [15]:
history = model.fit(train_data, validation_data=val_data, epochs=10)


Epoch 1/10


ValueError: Creating variables on a non-first call to a function decorated with tf.function.

In [ ]:
import tensorflow as tf

# Clear any previous session
tf.keras.backend.clear_session()

# Build the model again
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model
history = model.fit(train_data, validation_data=val_data, epochs=10)



Epoch 1/10


C:\Users\Saurav\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


525/525 ━━━━━━━━━━━━━━━━━━━━ 317s 597ms/step - accuracy: 0.9870 - loss: 0.0154 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
222/525 ━━━━━━━━━━━━━━━━━━━━ 2:02 405ms/step - accuracy: 1.0000 - loss: 0.0000e+00